Must needed setup before every assignment

In [0]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 60kB/s 
     |████████████████████████████████| 204kB 43.2MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=bdc4e84b81b4a30e44b99e0d3654ea79f852adb5630957c5d7611e779e8aaabf
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Imports 

In [0]:
import itertools
from pyspark import SparkContext, SparkConf
from pyspark.sql import *

Setup Spark

In [0]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

Check if Spark is working

In [0]:
spark

Variables

In [0]:
#dataPath = 'graph-small.txt' # node # 1 ~ 100 
dataPath = 'graph-full.txt' # node # 1 ~ 1000
# check min and max node #
mi = float('inf')
Mx = -float('inf')

Open file

In [0]:
with open(dataPath) as f:
    for line in f:
        a, b = map(int, line.split())
        mi = min(mi, a, b)
        Mx = max(Mx, a, b)

n = Mx

lines = sc.textFile(dataPath)
edges = lines.map(lambda line: list(map(int, line.split())))

Definitions

In [0]:
def aggregateUV(st, v):
    st.add(v)
    return st

def aggregateUU(st1, st2):
    return st1 | st2

Get Edges

In [0]:
u_outNeigs = edges.aggregateByKey(set(), aggregateUV, aggregateUU)
# u_outDegs = u_outNeigs.map(lambda u_st: (u_st[0], len(u_st[1])))

M = {}
for u, outNeigs in u_outNeigs.collect():
    M[u] = outNeigs

u_outNeigs = sc.broadcast(M)

beta = 0.8
one_beta_n = (1 - beta) / n
numIters = 40 #Number of iterations

Last definiton

In [0]:
def Mr(r):
    M = u_outNeigs.value
    new = [0] * n
    for j, rj in enumerate(r, 1): # 0/1 indexing
        if rj == 0:
            continue
        # assert deg != 0
        inv = rj / len(M[j])
        for i in M[j]:
            new[i - 1] += inv # 0/1 indexing
    return new

Output

In [0]:
r = [1 / n] * n


for _ in range(numIters):
    r = [one_beta_n + beta * val for val in Mr(r)]

r_id = zip(r, range(1, n + 1))
r_id = sorted(r_id, reverse=True)
r_id_reverse = r_id[::-1]

print(r_id[:5])
print(r_id_reverse[:5])

[(0.0020202911815182184, 263), (0.0019433415714531497, 537), (0.0019254478071662631, 965), (0.001852634016241731, 243), (0.0018273721700645144, 285)]
[(0.0003286018525215297, 558), (0.0003513568937516577, 93), (0.00035314810510596274, 62), (0.0003548153864930145, 424), (0.00038779848719291705, 408)]
